In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
import cmath
from numba import njit, f8,i8,c16
import numba
import matplotlib.animation as animation
from IPython.display import HTML
import copy

import sys
sys.path.append('../../lb')
import RungeKutta_acceralated
import PCA

#パラメーター
nu = 0.001

f = (1 + 1j) * 5 * 0.001

Dimention = 14 #uの次元数
q = 2
k_0 = 2 ** (-4)
k_n = np.zeros((Dimention+4,1))
k_n[2:-2,0] = [k_0 * q ** (i+1) for i in range(Dimention)]
#print(k_n)

c_n_1 = copy.deepcopy(k_n)[2:-2, 0:]
c_n_1[-2:, 0:] = np.zeros((2,1))
#print(c_n_1)

beta = 0.5
c_n_2 = copy.deepcopy(k_n)[1:-3, 0:] * -beta
c_n_2[-1, 0] = np.zeros((1, 1))
#print(c_n_2)

c_n_3 = copy.deepcopy(k_n)[:-4, 0:] * (beta - 1)
#print(c_n_3)

#parameter = [nu,f,k_n,c_n_1,c_n_2,c_n_3]
parameter = {
    'nu' : nu,
    'f' : f,
    'k_n' : k_n,
    'c_n_1' : c_n_1,
    'c_n_2' : c_n_2,
    'c_n_3' : c_n_3
}
#print(parameter)
#print(k_n)

runge_para = {
    'start' :  np.array([
        [0.4350E+00 + 0.5008E+00j],
        [0.1259E+00 + 0.2437E+00j],
        [-0.8312E-01 + -0.4802E-01j],
        [0.5164E-01 + -0.1599E+00j],
        [-0.1899E+00 + -0.3602E-01j],
        [0.4093E-03 + 0.8506E-01j],
        [0.9539E-01 + 0.3215E-01j],
        [-0.5834E-01 + 0.4433E-01j],
        [-0.8790E-02 + 0.2502E-01j],
        [0.3385E-02 + 0.1148E-02j],
        [-0.7072E-04 + 0.5598E-04j],
        [-0.5238E-07 + 0.1467E-06j],
        [0.1E-07 + 0.1E-06j],
        [0.1E-07 + 0.1E-06j]
    ]),
    'step' : 0.005,
    'period' : (0,5000),
    'nu' : nu,
    'f' : f,
    'k_n' : k_n,
    'c_n_1' : c_n_1,
    'c_n_2' : c_n_2,
    'c_n_3' : c_n_3
}

In [ ]:
laminar = np.load("/Users/hibiki/Downloads/beta0.423_nu0.00017584784643038092_step0.01_1500check_500pro_200000period.npz")["laminar"]
cut = RungeKutta_acceralated.Long_laminar.loc_max_7(laminar, 4)


# ポアンカレ写像 (numpyロード)

#時間を指定して区切る
@njit
def trimmer(array,period,trimmered):
    for i in range(len(array[0,:])):
        if (array[-1,i] > period[0] and array[-1,i] < period[1]):
            trimmered = np.append(trimmered,array[:,i:i+1],axis = 1)
    return trimmered

In [ ]:
#ポアンカレ写像のプロット(ラミナーの時間、出したい変数を入力)

#プロットする期間たち
periods = np.array([[10000,14000],
                    [87000,94000],
                    [135000,137000],
                    [160000,163000],
                    [175000,181000],
                    [189000,192000],
                    [194000,197000],
                    [206000,207000],
                    [228000,229000],
                    [234000,236000],
                    [262000,266000],
                    [275000,282000],
                    [288000,296000],
                    [309000,310000],
                    [322000,325000],
                    [338000,341000],
                    [346000,348000],
                    [368000,371000],
                    [380000,382000],
                    [405000,409000],
                    [412000,416000],
                    [420000,422000],
                    [459000,465000],
                    [480000,486000],
                    [487000,489000],
                    [522000,524000],
                    [538000,541000],
                    [605000,606000],
                    [624000,629000],
                    [640000,643000],
                    [646000,650000],
                    [652000,653000],
                    [659000,662000],
                    [681000,691000],
                    [707000,711000],
                    [723000,724000],
                    [726000,727000],
                    [759000,760000],
                    [763000,768000],
                    [773000,774000],
                    [793000,795000],
                    [820000,824000],
                    [825000,827000],
                    [832000,833000],
                    [838000,840000],
                    [843000,850000],
                    [851000,859000],
                    [872000,875000],
                    [896000,897000],
                    [904000,908000],
                    [909000,915000],
                    [921000,923000],
                    [926000,927000],
                    [942000,943000],
                    [977000,983000],
                    [992000,993000]
                   ])

#プロットする変数
var1 = 4
var2 = 3

#プロットの設定
plt.figure(figsize=(10,10))
plt.xlabel(f"|u{var1}|")
plt.ylabel(f"|u{var2}|")
plt.xticks(np.arange(-0.6,0.6,step=0.02))
plt.xticks(rotation=90)
plt.yticks(np.arange(-0.4,0.4,step=0.02))
plt.grid(True)
plt.title(f"U_{4} is local maximum \n beta = {beta} \n nu = {nu}")

#plt.scatter(cut_array[var1-1,:],cut_array[var2-1,:],s=5,c="blue",label="all", alpha = 0.5＃plt.scatter(cut_array[var1-1,:],cut_array[var2-1,:],s=5,c="blue",label="all", alpha = 0.5)

for i in range(len(periods[:,0])):
    period = periods[i,:]
    trimmered = np.zeros((15,1))
    trimmered = trimmer(cut_array,period,trimmered)
    trimmered = np.delete(trimmered,0,1)
    if i == 0:
        plt.scatter(trimmered[var1-1,:],trimmered[var2-1,:],c="red",s=1,label="laminar")
    else:
        plt.scatter(trimmered[var1-1,:],trimmered[var2-1,:],c="red",s=1)
        
plt.scatter(cut[var1-1,:],cut[var2-1,:],s=10,c="green",label="laminar(beta=0.l423)", alpha = 0.5)
plt.legend()

plt.savefig(fname=f"../poincare/beta{0.l423}_nu{0.00017584784643038092}_{step}step_time{time}_{len(cut_array[:,0])}dim_u4_is_loc_max_U{var1}andU{var2}.png")

In [ ]:
#ポアンカレ写像のプロット(ラミナーの時間、出したい変数を入力)

#プロットする期間たち
periods = np.array([[10000,14000],
                    [87000,94000],
                    [135000,137000],
                    [160000,163000],
                    [175000,181000],
                    [189000,192000],
                    [194000,197000],
                    [206000,207000],
                    [228000,229000],
                    [234000,236000],
                    [262000,266000],
                    [275000,282000],
                    [288000,296000],
                    [309000,310000],
                    [322000,325000],
                    [338000,341000],
                    [346000,348000],
                    [368000,371000],
                    [380000,382000],
                    [405000,409000],
                    [412000,416000],
                    [420000,422000],
                    [459000,465000],
                    [480000,486000],
                    [487000,489000],
                    [522000,524000],
                    [538000,541000],
                    [605000,606000],
                    [624000,629000],
                    [640000,643000],
                    [646000,650000],
                    [652000,653000],
                    [659000,662000],
                    [681000,691000],
                    [707000,711000],
                    [723000,724000],
                    [726000,727000],
                    [759000,760000],
                    [763000,768000],
                    [773000,774000],
                    [793000,795000],
                    [820000,824000],
                    [825000,827000],
                    [832000,833000],
                    [838000,840000],
                    [843000,850000],
                    [851000,859000],
                    [872000,875000],
                    [896000,897000],
                    [904000,908000],
                    [909000,915000],
                    [921000,923000],
                    [926000,927000],
                    [942000,943000],
                    [977000,983000],
                    [992000,993000]
                   ])

#プロットの設定
plt.figure(figsize=(10,80))
plt.title(f"U_{4} is local maximum \n beta = {beta} \n nu = {nu}")
for i in range(len(cut_array[:,0])-1):
    plt.xlabel(f"u_{i}")
    plt.subplot(len(cut_array[:,0])-1,1,i+1)
    plt.scatter(cut_array[i,:],[0]*len(cut_array[i,:]),c="black",label="all")
    #メモリ幅
    if max(cut_array[i,:])>1:
        plt.xticks(np.arange(0,1.5,step=0.02))
        plt.tick_params(axis="x", labelrotation=90)
    
    elif max(cut_array[i,:])>0.5:
        plt.xticks(np.arange(0,1,step=0.02))
        plt.tick_params(axis="x", labelrotation=90)
        
    elif max(cut_array[i,:])>0.2:
        plt.xticks(np.arange(0,0.5,step=0.005))
        plt.tick_params(axis="x", labelrotation=90,labelsize='small')
        
    elif max(cut_array[i,:])>0.1:
        plt.xticks(np.arange(0,0.2,step=0.004))
        plt.tick_params(axis="x", labelrotation=90)
        
    elif max(cut_array[i,:])>0.05:
        plt.xticks(np.arange(0,0.1,step=0.002))
        plt.tick_params(axis="x", labelrotation=90)
        
    elif max(cut_array[i,:])>0.01:
        plt.xticks(np.arange(0,0.05,step=0.0005))
        plt.tick_params(axis="x", labelrotation=90)
        
    elif max(cut_array[i,:])>0.003:
        plt.xticks(np.arange(0,0.01,step=0.0001))
        plt.tick_params(axis="x", labelrotation=90)
        
    elif max(cut_array[i,:])>0.00001:
        plt.xticks(np.arange(0,0.003,step=0.000005))
        plt.tick_params(axis="x", labelrotation=90)
    
    for j in range(len(periods[:,0])):
        period = periods[j,:]
        trimmered = np.zeros((15,1))
        trimmered = trimmer(cut_array,period,trimmered)
        trimmered = np.delete(trimmered,0,1)
        if j == 0:
            plt.scatter(trimmered[i,:],[0]*len(trimmered[i,:]),c="magenta",s=1,label="laminar")
        else:
            plt.scatter(trimmered[i,:],[0]*len(trimmered[i,:]),c="magenta",s=1)
    plt.grid(True)
    plt.legend()

plt.savefig(fname=f"../poincare/beta{beta}_nu{nu}_{step}step_time{time}_{len(cut_array[:,0])}dim_u4_is_loc_max_laminar領域設定用.png")

# 極大値の時でポアンカレ写像(軌道計算)

In [ ]:
#パラメータから軌道計算
runge_para["period"]=(0,400000)

runge_para['start'] = np.load("../laminar_initials/beta0.423_nu0.00017584784643038092_step0.01_1500check_500pro_20000period.npz")["start"]
#刻み幅
runge_para["step"] = 0.01

#nu
runge_para["nu"] = 0.00017584784643038092
#betaの値に従いc_n_2,c_n_3を更新
beta= 0.423

#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))
#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)

#インスタンス化
model = RungeKutta_acceralated.RungeKutta()
model.get_arr_latter(**runge_para,latter=1)

loc_max_of_Nth = 4
orbit_loc_max = RungeKutta_acceralated.Long_laminar.loc_max_7(model.arr_latter, loc_max_of_Nth)
laminar_loc_max = RungeKutta_acceralated.Long_laminar.loc_max_7(laminar, loc_max_of_Nth)

#プロットする変数
var1 = 3
var2 = 4


#プロットの設定
plt.figure(figsize=(10,10))
plt.xlabel(f"|u{var1}|")
plt.ylabel(f"|u{var2}|")

plt.xticks(rotation=90)
#plt.grid(True)
plt.title(f"U_{loc_max_of_Nth} is local maximum \n beta = {beta} \n nu = {runge_para['nu']}")



plt.scatter(orbit_loc_max[var1-1,:],orbit_loc_max[var2-1,:],s=2,c="blue",label="all")
plt.scatter(laminar_loc_max[var1-1,:],laminar_loc_max[var2-1,:],c="red",s=2,label="laminar")
  
plt.legend()

plt.savefig(fname=f"../poincare/beta{beta}_nu{nu}_{runge_para['step']}step_time{runge_para['period'][1]}_{model.arr_latter.shape[0]}dim_u{loc_max_of_Nth}_is_loc_max_U{var1}andU{var2}.png")

In [ ]:
#パラメータから軌道計算
runge_para["period"]=(0,400000)

runge_para['start'] = np.load("../laminar_initials/beta0.423_nu0.00017584784643038092_step0.01_1500check_500pro_20000period.npz")["start"]
#刻み幅
runge_para["step"] = 0.01

#nu
runge_para["nu"] = 0.00017584784643038092
#betaの値に従いc_n_2,c_n_3を更新
beta= 0.423

#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))
#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)

#インスタンス化
model = RungeKutta_acceralated.RungeKutta()
# model.get_arr_latter(**runge_para,latter=1)

# loc_max_of_Nth = 4
# orbit_loc_max = RungeKutta_acceralated.Long_laminar.loc_max_7(model.arr_latter, loc_max_of_Nth)
# laminar_loc_max = RungeKutta_acceralated.Long_laminar.loc_max_7(laminar, loc_max_of_Nth)

#プロットする変数
var1 = 3
var2 = 4


#プロットの設定
plt.figure(figsize=(10,10))
plt.xlabel(r"$|U_3|$")
plt.ylabel(r"$|U_4|")



plt.scatter(orbit_loc_max[var1-1,:],orbit_loc_max[var2-1,:],s=2,c="red",label="All")
plt.scatter(laminar_loc_max[var1-1,:],laminar_loc_max[var2-1,:],c="blue",s=2,label="Laminar")
  
plt.legend()

plt.savefig(fname=f"../poincare/beta{beta}_nu{nu}_{runge_para['step']}step_time{runge_para['period'][1]}_{model.arr_latter.shape[0]}dim_u{loc_max_of_Nth}_is_loc_max_U{var1}andU{var2}.png")

In [ ]:
fig, axes = plt.subplots(7,2, squeeze=False, figsize = (30, 100))
for i in range(1,15):
    loc_max_of_Nth = i
    orbit_loc_max = RungeKutta_acceralated.Long_laminar.loc_max_7(model.arr_latter, loc_max_of_Nth)
    laminar_loc_max = RungeKutta_acceralated.Long_laminar.loc_max_7(laminar, loc_max_of_Nth)

    #プロットする変数
    var1 = 3
    var2 = 4


    #プロットの設定
    axes[(i-1)//2, (i-1)%2].set_xlabel(f"|u{var1}|")
    axes[(i-1)//2, (i-1)%2].set_ylabel(f"|u{var2}|")
    #ax[i].grid(True)
    axes[(i-1)//2, (i-1)%2].set_title(f"U_{loc_max_of_Nth} is local maximum \n beta = {beta} \n nu = {runge_para['nu']}")



    axes[(i-1)//2, (i-1)%2].scatter(orbit_loc_max[var1-1,:],orbit_loc_max[var2-1,:],s=2,c="blue",label="all")
    axes[(i-1)//2, (i-1)%2].scatter(laminar_loc_max[var1-1,:],laminar_loc_max[var2-1,:],c="red",s=2,label="laminar")
  
    axes[(i-1)//2, (i-1)%2].legend()
fig.savefig(fname=f"../poincare/tile_beta{beta}_nu{nu}_{runge_para['step']}step_time{runge_para['period'][1]}_{model.arr_latter.shape[0]}dim_U{var1}andU{var2}.png")

In [ ]:
loc_max_of_Nth = 4
    l417_loc_max = RungeKutta_acceralated.Long_laminar.loc_max_7(np.load("/Users/hibiki/Downloads/beta0.417_nu0.00017632638671047897_step0.01_1500check_500pro_20000period.npz")["laminar"], loc_max_of_Nth)
    l418_loc_max = RungeKutta_acceralated.Long_laminar.loc_max_7(np.load("/Users/hibiki/Downloads/beta0.418_nu0.00017256_step0.01_2000check_500pro_40000period.npz")["laminar"], loc_max_of_Nth)
    l423_loc_max = RungeKutta_acceralated.Long_laminar.loc_max_7(np.load("/Users/hibiki/Downloads/beta0.423_nu0.00017584784643038092_step0.01_2000check_500pro_50000period.npz")["laminar"], loc_max_of_Nth)
    
    #プロットする変数
    var1 = 3
    var2 = 4

In [ ]:
fig, axes = plt.subplots(7,2, squeeze=False, figsize = (30, 100))
for i in range(1,15):
    loc_max_of_Nth = i
    l417_loc_max = RungeKutta_acceralated.Long_laminar.loc_max_7(np.load("/Users/hibiki/Downloads/beta0.417_nu0.00017632638671047897_step0.01_1500check_500pro_20000period.npz")["laminar"], loc_max_of_Nth)
    l418_loc_max = RungeKutta_acceralated.Long_laminar.loc_max_7(np.load("/Users/hibiki/Downloads/beta0.418_nu0.00017256_step0.01_2000check_500pro_40000period.npz")["laminar"], loc_max_of_Nth)
    l423_loc_max = RungeKutta_acceralated.Long_laminar.loc_max_7(np.load("/Users/hibiki/Downloads/beta0.423_nu0.00017584784643038092_step0.01_2000check_500pro_50000period.npz")["laminar"], loc_max_of_Nth)
    
    #プロットする変数
    var1 = 3
    var2 = 4


    #プロットの設定
    axes[(i-1)//2, (i-1)%2].set_xlabel(f"|u{var1}|")
    axes[(i-1)//2, (i-1)%2].set_ylabel(f"|u{var2}|")
    #ax[i].grid(True)
    axes[(i-1)//2, (i-1)%2].set_title(f"U_{loc_max_of_Nth} is local maximum \n beta = {beta} \n nu = {runge_para['nu']}")



    axes[(i-1)//2, (i-1)%2].scatter(l417_loc_max[var1-1,:],l417_loc_max[var2-1,:],s=2,c="blue",label="l417")
    axes[(i-1)//2, (i-1)%2].scatter(l418_loc_max[var1-1,:],l418_loc_max[var2-1,:],c="red",s=2,label="l418")
    axes[(i-1)//2, (i-1)%2].scatter(l423_loc_max[var1-1,:],l423_loc_max[var2-1,:],c="green",s=2,label="l423")
    
    axes[(i-1)//2, (i-1)%2].legend()

plt.savefig(fname=f"../poincare/tile_laminar_beta0.417_0.418_0.423_14dim_u4_is_loc_max_U3andU4.png")

# extract

In [ ]:
def extracter(data, var1, var2, var1_s, var1_e, var2_s, var2_e):
    condition = np.where((var1_e >= data[var1-1, :]) & (data[var1-1, :] >= var1_s) & (var2_e >= data[var2-1, :]) & (data[var2-1, :] >= var2_s))
    result = data[:, condition[0]]
    return result
fig, axes = plt.subplots(figsize = (10, 10))
loc_max_of_Nth = 4
l417_loc_max = RungeKutta_acceralated.Long_laminar.loc_max_7(np.load("/Users/hibiki/Downloads/beta0.417_nu0.00017632638671047897_step0.01_1500check_500pro_20000period.npz")["laminar"], loc_max_of_Nth)
l418_loc_max = RungeKutta_acceralated.Long_laminar.loc_max_7(np.load("/Users/hibiki/Downloads/beta0.418_nu0.00017256_step0.01_2000check_500pro_40000period.npz")["laminar"], loc_max_of_Nth)
l423_loc_max = RungeKutta_acceralated.Long_laminar.loc_max_7(np.load("/Users/hibiki/Downloads/beta0.423_nu0.00017584784643038092_step0.01_2000check_500pro_50000period.npz")["laminar"], loc_max_of_Nth)

#プロットする変数
var1 = 3
var2 = 4


#プロットの設定
axes.set_xlabel(r"$|U_3|$")
axes.set_ylabel(r"$|U_4|$")
#axes.set_title(f"U_{loc_max_of_Nth} is local maximum \n beta = {beta} \n nu = {runge_para['nu']}")

# l417_loc_max=extracter(l417_loc_max, 3, 4, 0.1, 0.3, 0, 0.25)
# l418_loc_max=extracter(l418_loc_max, 3, 4, 0.1, 0.3, 0, 0.25)
# l423_loc_max=extracter(l423_loc_max, 3, 4, 0.1, 0.3, 0, 0.25)

axes.scatter(l417_loc_max[var1-1,:],l417_loc_max[var2-1,:],s=2,c="blue",label=r"$\beta = 0.417$")
axes.scatter(l418_loc_max[var1-1,:],l418_loc_max[var2-1,:],c="red",s=2,label=r"$\beta = 0.418$")
axes.scatter(l423_loc_max[var1-1,:],l423_loc_max[var2-1,:],c="green",s=2,label=r"$\beta = 0.423$")

axes.legend()

plt.savefig(fname=f"../poincare/laminar_beta0.417_0.418_0.423_14dim_u4_is_loc_max_U3andU4.png")

In [ ]:
laminar = model.arr_latter[:,:model.arr_latter.shape[1]//2+1]

array = np.zeros((laminar.shape[0],laminar.shape[1]*2-1), dtype=np.float64)
array[:, 0] = np.real(laminar[:, -1])
array[:,1::2] = np.real(laminar[:,:-1])
array[:,2::2] = np.imag(laminar[:,:-1])
np.savetxt("orbit_200000period_beta0.l417_nu0.00017322774042408294_step0.01.txt", array)

# PCAをやってみる(領域を定めるときにできるだけ簡便にしたいから)

In [ ]:
lowerdim = 3
# 2. 主成分分析による次元削減
PCAmodel = PCA.PCA(cut_array[:-1,:].T)
PCAmodel.reduceDim(lowerDim=lowerdim)
#-------------------

#-------------------
# 3. モデルパラメータの表示
print(f"固有値:\nlambda={PCAmodel.L}")
#print(f"正規直交基底ベクトル:\nw=\n{PCAmodel.W}")
#-------------------

#-------------------
# 4. データと主成分軸のプロット
# 次元削減後のデータ（主成分得点）のプロット
if PCAmodel.lowerDim == 2:
    PCAmodel.plotResult()
    
# # 主成分軸（平面）のプロット
# if (PCAmodel.lowerDim == 2):
#     PCAmodel.plotModel3D(xLabel="1st principle component",yLabel=2nd principle component,zLabel=3rd principle component,nGrids=20,
#         fName=f"../results/PCA_result_plane.png")
#-------------------

#-------------------
# 5. モデルの評価
# 寄与率と累積寄与率の計算
contRatio,cumContRatio = PCAmodel.compContRatio()
print(f"寄与率:{np.round(contRatio,decimals=1)}")
#print(f"累積寄与率:{np.round(cumContRatio,decimals=1)}")

# 主成分負荷量の計算
#print(f"主成分負荷量:\n{np.round(PCAmodel.compLoading(),decimals=1)}")
#-------------------


In [ ]:
cut_pca = np.concatenate((PCAmodel.F[:,:lowerdim].T,cut_array[-1:,:]),axis = 0)

if lowerdim == 1:
    plt.figure(figsize=(10,10),dpi=300)
    plt.xlabel(f"1st principle component")
    plt.ylabel(f"")
    plt.title(f"U_{4} is local maximum \n beta = {beta} \n nu = {nu}")
    plt.xticks(np.arange(-0.6,0.6,step=0.02))
    plt.xticks(rotation=90)
    plt.yticks(np.arange(-0.4,0.4,step=0.02))
    plt.grid(True)
    plt.scatter(cut_pca[0,:],[[0]*len(cut_pca[0,:])],s=0.1,c="blue",label="all")

    for i in range(len(periods[:,0])):
        period = periods[i,:]
        trimmered = np.zeros((2,1))
        trimmered = trimmer(cut_pca,period,trimmered)
        trimmered = np.delete(trimmered,0,1)
        if i == 0:
            plt.scatter(trimmered[0,:],[[0]*len(trimmered[0,:])],c="red",s=0.1,label="laminar")
        else:
            plt.scatter(trimmered[0,:],[[0]*len(trimmered[0,:])],c="red",s=0.1)
    plt.legend()

elif lowerdim == 2:
    plt.figure(figsize=(10,10),dpi=300)
    plt.xlabel(f"1st principle component")
    plt.ylabel(f"2nd principle component")
    plt.title(f"U_{4} is local maximum \n beta = {beta} \n nu = {nu}")
    plt.xticks(np.arange(-0.6,0.6,step=0.02))
    plt.xticks(rotation=90)
    plt.yticks(np.arange(-0.4,0.4,step=0.02))
    plt.grid(True)
    plt.scatter(cut_pca[0,:],cut_pca[1,:],s=5,c="blue",label="all")

    for i in range(len(periods[:,0])):
        period = periods[i,:]
        trimmered = np.zeros((3,1))
        trimmered = trimmer(cut_pca,period,trimmered)
        trimmered = np.delete(trimmered,0,1)
        if i == 0:
            plt.scatter(trimmered[0,:],trimmered[1,:],c="red",s=5,label="laminar")
        else:
            plt.scatter(trimmered[0,:],trimmered[1,:],c="red",s=5)
    plt.legend()
    
else:
    fig = plt.figure(figsize=(10,30))

    ax1 = fig.add_subplot(3,1,1)
    ax1.set_xlabel("1st principle component")
    ax1.set_ylabel("2nd principle component")
    ax1.set_title(f"U_{4} is local maximum \n beta = {beta} \n nu = {nu}")
    ax1.set_xticks(np.arange(-0.6,0.6,step=0.02))
    ax1.tick_params(axis="x", labelrotation=90)
    ax1.set_yticks(np.arange(-0.4,0.4,step=0.02))
    ax1.grid(True)
    
    ax1.scatter(cut_pca[0,:],cut_pca[1,:],s=5,c="blue",label="all")

    for i in range(len(periods[:,0])):
        period = periods[i,:]
        trimmered = np.zeros((lowerdim+1,1))
        trimmered = trimmer(cut_pca,period,trimmered)
        trimmered = np.delete(trimmered,0,1)
        if i == 0:
            ax1.scatter(trimmered[0,:],trimmered[1,:],c="red",s=5,label="laminar")
        else:
            ax1.scatter(trimmered[0,:],trimmered[1,:],c="red",s=5)
            
    ax2 = fig.add_subplot(3,1,2)
    ax2.set_xlabel("2nd principle component")
    ax2.set_ylabel("3rd principle component")
    ax2.set_title(f"U_{4} is local maximum \n beta = {beta} \n nu = {nu}")
    ax2.set_xticks(np.arange(-0.4,0.4,step=0.02))
    ax2.tick_params(axis="x", labelrotation=90)
    ax2.set_yticks(np.arange(-0.2,0.2,step=0.02))
    ax2.grid(True)
    
    ax2.scatter(cut_pca[1,:],cut_pca[2,:],s=5,c="blue",label="all")

    for i in range(len(periods[:,0])):
        period = periods[i,:]
        trimmered = np.zeros((lowerdim+1,1))
        trimmered = trimmer(cut_pca,period,trimmered)
        trimmered = np.delete(trimmered,0,1)
        if i == 0:
            ax2.scatter(trimmered[1,:],trimmered[2,:],c="red",s=5,label="laminar")
        else:
            ax2.scatter(trimmered[1,:],trimmered[2,:],c="red",s=5)
    
    ax3 = fig.add_subplot(3,1,3)
    ax3.set_xlabel("3rd principle component")
    ax3.set_ylabel("2nd principle component")
    ax3.set_title(f"U_{4} is local maximum \n beta = {beta} \n nu = {nu}")
    ax3.set_xticks(np.arange(-0.2,0.2,step=0.02))
    ax3.tick_params(axis="x", labelrotation=90)
    ax3.set_yticks(np.arange(-0.6,0.6,step=0.02))
    ax3.grid(True)
    
    ax3.scatter(cut_pca[2,:],cut_pca[0,:],s=5,c="blue",label="all")

    for i in range(len(periods[:,0])):
        period = periods[i,:]
        trimmered = np.zeros((lowerdim+1,1))
        trimmered = trimmer(cut_pca,period,trimmered)
        trimmered = np.delete(trimmered,0,1)
        if i == 0:
            ax3.scatter(trimmered[2,:],trimmered[0,:],c="red",s=5,label="laminar")
        else:
            ax3.scatter(trimmered[2,:],trimmered[0,:],c="red",s=5)
            
# else:
#     fig = plt.figure(figsize=(10,15))

#     ax = fig.add_subplot(projection='3d')
#     ax.set_xlabel("1st principle component")
#     ax.set_ylabel("2nd principle component")
#     ax.set_zlabel("3rd principle component")
#     ax.set_title(f"U_{4} is local maximum \n beta = {beta} \n nu = {nu}")
    
#     ax.scatter(cut_pca[0,:],cut_pca[1,:],cut_pca[2,:],s=5,c="blue",label="all")

#     for i in range(len(periods[:,0])):
#         period = periods[i,:]
#         trimmered = np.zeros((lowerdim+1,1))
#         trimmered = trimmer(cut_pca,period,trimmered)
#         trimmered = np.delete(trimmered,0,1)
#         if i == 0:
#             ax.scatter(trimmered[0,:],trimmered[1,:],trimmered[2,:],c="red",s=0.1,label="laminar")
#         else:
#             ax.scatter(trimmered[0,:],trimmered[1,:],trimmered[2,:],c="red",s=1)
#     plt.legend()

plt.savefig("pca_poincare.png")


# ポアンカレ写像
## u4で切った時の他の2変数の値

In [ ]:
#u_4が閾値を下から通る時のu_nをndarrayで返す関数
@njit(f8[:](c16[:,:],i8,i8,f8[:],f8),cache=True)
def cutter(arr,step_number,dimention,z_list,cutter_num):
    for j in range(len(arr[0,:]) - 1):
        if abs(arr[3,j]) < cutter_num and abs(arr[3,j+1]) > cutter_num:
            z_list = np.append(z_list,abs(arr[dimention-1,j+1]))
    return z_list

In [ ]:
%%time
runge_para["period"]=(0,30000)

latter = 10

#プロットする変数
var1 = 5
var2 = 6

#第四変数を切る数
cutting_num = 0.18

#nu
runge_para["nu"] = 0.0001746309482023122
#betaの値に従いc_n_2,c_n_3を更新
beta=  0.41404040404040404

#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)

step_number = int((runge_para["period"][1] - runge_para["period"][0]) // runge_para["step"]) + 1

#インスタンス化
model = RungeKutta_acceralated.RungeKutta()
model.get_arr_latter(**runge_para,latter=latter)

#プロットの設定
plt.figure(figsize=(10,10))
plt.xlabel(f"|u{var1}|")
plt.ylabel(f"|u{var2}|")
plt.title(f"U_{4} ={cutting_num} \n beta = {beta} \n nu = {runge_para['nu']} \n period({int(abs(model.arr_latter[-1,0]))},{int(abs(model.arr_latter[-1,-1]))})")

x_list=np.array([],dtype=np.float64)#var1を入れる空のリスト
y_list=np.array([],dtype=np.float64)#var2を入れる空のリスト

x_list = cutter(model.arr_latter,step_number,var1,x_list,cutting_num)
y_list = cutter(model.arr_latter,step_number,var2,y_list,cutting_num)

plt.scatter(x_list, y_list,s=10)

plt.savefig(fname=f"../poincare/beta{beta}_nu{runge_para['nu']}_{runge_para['step']}step_time({int(abs(model.arr_latter[-1,0])),int(abs(model.arr_latter[-1,-1]))})_{Dimention}dim_u4={cutting_num}_U{var1}andU{var2}.png")

##　第N変数とその時間遅れ

In [ ]:
#u_Nが閾値を下から通る時のu_nをndarrayで返す関数(delayは刻み幅ですでに割られてる)
@njit(f8[:](c16[:,:],f8[:],i8,i8,f8,i8),cache=True)
def cutter_delay(arr,z_list,step_number,dimention,cutter_num,delay):
    for j in range(delay//step_number,len(arr[0,:]) - 1):
        if abs(arr[dimention-1,j]) < cutter_num and abs(arr[dimention-1,j+1]) > cutter_num:
            z_list = np.append(z_list,abs(arr[dimention-1,j+1-delay]))
    return z_list

In [ ]:
%%time
runge_para["period"]=(0,35000)

latter = 10

#時間遅れ(秒)
delay = 1

#第N変数
N = 4
#第N変数を切る数
cutting_num = 0.3

#nu
runge_para["nu"] = 0.0001746309482023122
#betaの値に従いc_n_2,c_n_3を更新
beta=  0.41404040404040404

#c_n_2
runge_para["c_n_2"] = runge_para["k_n"][1:-3, 0:] * -beta
runge_para["c_n_2"][-1, 0:] = np.zeros((1, 1))

#c_n_3
runge_para["c_n_3"] = runge_para["k_n"][:-4, 0:] * (beta - 1)

step_number = int((runge_para["period"][1] - runge_para["period"][0]) // runge_para["step"]) + 1

#インスタンス化
#model = RungeKutta_acceralated.RungeKutta()
#model.get_arr_latter(**runge_para,latter=latter)

#プロットの設定
plt.figure(figsize=(10,10))
plt.xlabel(f"t-$\u03c4$")
plt.ylabel(f"t-2$\u03c4$")
plt.title(f"U_{N}(t) ={cutting_num} \n beta = {beta} \n nu = {runge_para['nu']} \n period({int(abs(model.arr_latter[-1,0]))},{int(abs(model.arr_latter[-1,-1]))}) \n $\u03c4$={delay}")

x_list=np.array([],dtype=np.float64)#tau時間遅れを入れる空のリスト
y_list=np.array([],dtype=np.float64)#2tau時間遅れを入れる空のリスト

x_list = cutter_delay(model.arr_latter,x_list,step_number,N,cutting_num,delay//runge_para['step'])
y_list = cutter_delay(model.arr_latter,y_list,step_number,N,cutting_num,2*delay//runge_para['step'])

plt.scatter(x_list, y_list,s=10)

plt.savefig(fname=f"../poincare/beta{beta}_nu{runge_para['nu']}_{runge_para['step']}step_time({int(abs(model.arr_latter[-1,0])),int(abs(model.arr_latter[-1,-1]))})_{Dimention}dim_u{N}={cutting_num}.png")